# Challenge 4: Telemetry to the Rescue

## Find crash in telemetry data

You need to design a SQL query that returns the `car_number`, `driver_name` and the average `brake` & `speed` for the full second just before the crash. You should have the timestamp of the crash from the previous challenge.

> **Keep in mind that the timestamp is in local time (UTC + 2), whereas the telemetry data uses UTC.**

In [ ]:
# Find the crash in telemetry data that is loaded into BQ. Add your SQL query below
%%bigquery telemetry_during_crash

DECLARE crash_timestamp TIMESTAMP DEFAULT '2024-05-11 13:42:06 UTC';
-- Remember that the CCTV footage is in local time (CEST, which is UTC+2) whereas the telemetry data is in UTC.

SELECT
    driver_name,
    ROUND(AVG(tv_speed),3) AS avg_speed_last_second,
    ROUND(AVG(tv_brake),3) AS avg_brake_last_second
FROM
    `concise-mesh-477218-a5.dataset_name.telemetry` # Replace with yours. Ex: GOOGLE_CLOUD_PROJECT.DATASET_NAME.telemetry
WHERE
    time_utc >= TIMESTAMP_SUB(crash_timestamp, INTERVAL 1 SECOND)
    AND time_utc <= crash_timestamp
GROUP BY
    driver_name

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
print(telemetry_during_crash.to_markdown())

|    | driver_name             |   avg_speed_last_second |   avg_brake_last_second |
|---:|:------------------------|------------------------:|------------------------:|
|  0 | ARON, Paul              |                  66.551 |                   0     |
|  1 | KING, Jordan            |                 165.46  |                   0     |
|  2 | BARNARD, Taylor         |                 176.806 |                   0     |
|  3 | ROWLAND, Oliver         |                 128.462 |                   1.05  |
|  4 | DA COSTA, António Félix |                 136.381 |                   0     |
|  5 | FENESTRAZ, Sacha        |                  92.994 |                   2     |
|  6 | GÜNTHER, Maximilian     |                  64.928 |                  98     |
|  7 | WEHRLEIN, Pascal        |                 145.025 |                   0     |
|  8 | NATO, Norman            |                  50.598 |                   2.444 |
|  9 | CASSIDY, Nick           |                  49.758 |       

## Find the drivers

Now we have the filtered & aggregated data, let's ask Gemini to tell us which two drivers were involved in the crash.

Design a prompt that determines **which drivers** were involved and **why the model thinks that**.

> Note that we're *appending the result of the query from the previous cell to your prompt*, so you can refer to that.

In [11]:
import vertexai
from google import genai
from IPython.display import Markdown

PROJECT_ID = "concise-mesh-477218-a5" # Add your project ID here
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [9]:
prompt = """
Analyze this telemetry to determine the core drivers involved.

For this incident, identify the two drivers whose telemetry shows that they were involved in the accident.
Explain why.

Data:
{data}
"""

In [12]:

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents = prompt.format(data=telemetry_during_crash.to_markdown())
)

Markdown(response.text)

Based on the telemetry data, the two drivers involved in the accident are:

1.  **GÜNTHER, Maximilian**
2.  **HUGHES, Jake**

### Explanation

The key to identifying the involved drivers lies in the `avg_brake_last_second` column, which shows the intensity of braking in the final moment of data recording.

1.  **Extreme Braking Application:** Both Günther and Hughes show brake application values of **98** and **100**, respectively. These values are extreme outliers compared to the rest of the field. In motorsport, applying 98-100% brake pressure is an emergency action, indicating a driver is either trying to avoid an imminent collision or has lost control and is attempting to scrub off as much speed as possible before impact.

2.  **Comparison with Other Drivers:** Most other drivers have a brake application of 0, meaning they were either accelerating or coasting. A few drivers show very low values (e.g., Rowland at 1.05, Fenestraz at 2), which indicates light braking. The values for Günther and Hughes are drastically different, pointing to a sudden, violent event unique to them.

3.  **Associated Speed:** Their speeds (64.9 kph for Günther and 61.0 kph for Hughes) are relatively low compared to drivers who were clearly still at racing speed, but are consistent with a vehicle that is rapidly decelerating just before or during a crash.

In conclusion, the telemetry signature of simultaneous, maximum-force braking is the clearest indicator of an accident, making **Günther** and **Hughes** the core drivers involved.